In [1]:
# Import necessary packages
import opentrons.execute
import json

In [2]:
# Setup protocol and robot
protocol = opentrons.execute.get_protocol_api('2.12')
protocol.home()

Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.


In [3]:
# Set labware
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    plate_96 = protocol.load_labware_from_definition(labware_def, 2)
    plate_out = protocol.load_labware_from_definition(labware_def, 10)

with open ('custom_labware/enzymax_12_reservoir_20ml.json') as labware_file:
    labware_def = json.load(labware_file)
    liquids_reservoir = protocol.load_labware_from_definition(labware_def, 5)
    
tiprack_1000 = protocol.load_labware('opentrons_96_tiprack_1000ul', '6')
tiprack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', '3')

In [4]:
p1000 = protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tiprack_1000])
p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=[tiprack_300])

In [5]:
mag_module = protocol.load_module("magnetic module", '1')
with open('custom_labware/eppendorf_96_wellplate_2ml_deep.json') as labware_file:
    labware_def = json.load(labware_file)
    mag_plate = mag_module.load_labware_from_definition(labware_def, 1)

The labware definition for 1 does not define a default engagement height for use with the Magnetic Module; you must specify a height explicitly when calling engage().


In [6]:
samples = [plate_96['E1'], plate_96['E2'], plate_96['E3']]
lysis_buff = liquids_reservoir['A1']
neut_buff = liquids_reservoir['A2']
wash_buff = liquids_reservoir['A3']
ethanol = liquids_reservoir['A4']
elut_buff = liquids_reservoir['A5']
waste = liquids_reservoir['A12']

In [7]:
# Calibrate tiprack for p1000
tiprack_1000.set_offset(x=-0.6, y=0.0, z=0.)
p1000.move_to(tiprack_1000['A1'].top())

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [8]:
p1000.pick_up_tip()

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [12]:
# Calibrate remaining labware
plate_96.set_offset(x=-1, y=0.5, z=0.)
p1000.move_to(plate_96['A1'].top())

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [13]:
liquids_reservoir.set_offset(x=0.0, y=0.0, z=0.3)
p1000.move_to(liquids_reservoir['A1'].top())

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [16]:
mag_plate.set_offset(x=-0.7, y=0.4, z=0.)
p1000.move_to(mag_plate['A1'].top())

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [17]:
p1000.return_tip()

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [18]:
# Calibrate tiprack for p300
tiprack_300.set_offset(x=0.2, y=0.1, z=0.)
p300.move_to(tiprack_300['A1'].top())

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [20]:
for sample in samples:
    p300.transfer(120, lysis_buff, sample, mix_after = (5, 200))

protocol.delay(minutes = 4)

for sample in samples:
    p300.transfer(240, neut_buff, sample, mix_after = (5, 250))

# Centrifuge at max speed for 20 mins

KeyboardInterrupt: 

In [23]:
p1000.pick_up_tip()

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [26]:
# Calibrate labware again after centrifugation
plate_96.set_offset(x=-0.3, y=0.0, z=0.0) # might need to tweak height here
p1000.move_to(plate_96['A1'].top())

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [27]:
p1000.return_tip()

<InstrumentContext: p1000_single_v2.1 in LEFT>

In [28]:
# Aspirate slowly to avoid cell debris
for sample in samples:
    p1000.pick_up_tip()
    p1000.aspirate(300, sample.bottom(z=3.0), rate=0.2)
    p1000.dispense(300, mag_plate[sample.well_name])
    p1000.drop_tip()

In [29]:
for sample in samples:
    p1000.transfer(300, ethanol, mag_plate[sample.well_name], mix_after = (5, 600))

In [30]:
for i in range(4):
    if i > 0:
        protocol.delay(minutes = 3)
    for sample in samples:
        p1000.pick_up_tip()
        p1000.flow_rate.dispense = 800
        p1000.mix(5, 400, mag_plate[sample.well_name])
        p1000.move_to(mag_plate[sample.well_name].top())
        p1000.flow_rate.dispense = 300
        p1000.blow_out()
        p1000.drop_tip()
# wait 4 mins. repeat 4x (mix-w4-mix-w4-mix-w4-mix-w4-mix)

In [31]:
mag_module.engage(12)
protocol.delay(minutes = 2)

# Pick up supernatant slowly
for sample in samples:
    p1000.pick_up_tip()
    p1000.aspirate(700, mag_plate[sample.well_name], rate=0.2)
    p1000.dispense(700, waste)
    p1000.drop_tip()
mag_module.disengage()

In [32]:
for sample in samples:
    p1000.pick_up_tip()
    p1000.aspirate(500, wash_buff)
    p1000.dispense(500, mag_plate[sample.well_name])
    p1000.flow_rate.dispense = 800
    p1000.mix(5, 400, mag_plate[sample.well_name])
    p1000.move_to(mag_plate[sample.well_name].top())
    p1000.flow_rate.dispense = 300
    p1000.blow_out()
    p1000.drop_tip()

In [33]:
mag_module.engage(12)
protocol.delay(minutes = 2)

# Pick up supernatant slowly
for sample in samples:
    p1000.pick_up_tip()
    p1000.aspirate(500, mag_plate[sample.well_name], rate=0.2)
    p1000.dispense(500, waste)
    p1000.drop_tip()
mag_module.disengage()
# go dry at 65C for 20 mins

In [34]:
p300.pick_up_tip()

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [38]:
# Calibrate labware again
mag_plate.set_offset(x=-0.9, y=0.8, z=0.)
p300.move_to(mag_plate['A1'].top())

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [39]:
p300.return_tip()

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [40]:
for sample in samples:
    p300.transfer(100, elut_buff, mag_plate[sample.well_name], mix_after=(5, 80))

In [41]:
for i in range(4):
    if i > 0:
        protocol.delay(minutes = 1)
    for sample in samples:
        p300.pick_up_tip()
        p300.flow_rate.dispense = 800
        p300.mix(10, 80, mag_plate[sample.well_name])
        p300.move_to(mag_plate[sample.well_name].top())
        p300.flow_rate.dispense = 300
        p300.blow_out()
        p300.drop_tip()

In [42]:
mag_module.engage(12)
protocol.delay(minutes=2)

# Pick up supernatant slowly
for sample in samples:
    p300.pick_up_tip()
    p300.aspirate(100, mag_plate[sample.well_name], rate=0.2)
    p300.dispense(100, plate_out[sample.well_name])
    p300.drop_tip()

mag_module.disengage()

In [43]:
protocol.home()